# Homework 3

## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from collections import namedtuple
import itertools
import numpy as np


class RV:
  """A random variable with a finite domain.

  Example usage:
    A = RV("A", ["x", "y", "z"])
    print(A.domain)
    print(A.dim)
    B = RV("B", [(0, 0), (0, 1), (0, 2)]))
    print(B.domain)
    print(B.dim)
  """
  def __init__(self, name, domain):
    """Initialize a RV.

    Args:
      name: str name for the RV.
      domain: list or tuple of domain values.
    """
    assert isinstance(domain, (list, tuple))
    self.name = name
    self.domain = domain
    self.dim = len(domain)

  def __hash__(self):
    return hash((self.name, tuple(self.domain)))

  def __eq__(self, other):
    return self.name == other.name and self.domain == other.domain

  def __repr__(self):
    return f"RV('{self.name}', {self.domain})"


class Potential:
  """A potential over RVs.

  Example usage:
    A = RV("varA", ["x", "y", "z"])
    B = RV("varB", [0, 1])
    table = np.array([
      [0.1, 0.0],
      [0.4, 0.9],
      [0.5, 0.1]
    ])
    potential = Potential([A, B], table)
    print(potential.rvs)
    print(potential.get(("y", 0)))
    print(potential.get_by_rvs({A: "y", B: 0}))
    print(potential.get_by_names({"varA": "y", "varB": 0}))
  """
  def __init__(self, rvs, table):
    """Create a potential from a list of RVs and a numpy array.

    The order of the random variables corresponds to the axes
    of the numpy array.

    Args:
      rvs: A list or tuple of RVs.
      array: A numpy array of potential values.

    Returns:
      potential: A Potential."""
    assert isinstance(rvs, (tuple, list))
    assert len(rvs) == len(table.shape)
    assert all(rv.dim == dim for (rv, dim) in zip(rvs, table.shape))
    assert isinstance(table, np.ndarray)
    self.rvs = rvs
    self.table = table

  def set(self, assignment, new_value):
    """Given a complete assignment and a value, update table.

    Args:
      assignment: A tuple of values in the order of self.rv.
      new_value: A new value to add to the table.

    Returns:
      value: The value in self.table.
    """
    assert len(assignment) == len(self.rvs)
    indices = [None for _ in self.rvs]
    for index, value in enumerate(assignment):
      rv = self.rvs[index]
      indices[index] = rv.domain.index(value)
    self.table[tuple(indices)] = new_value

  def get(self, assignment):
    """Given a complete assignment of values, lookup table value.

    Args:
      assignment: A tuple of values in the order of self.rv.

    Returns:
      value: The value in self.table.
    """
    assert len(assignment) == len(self.rvs)
    indices = [None for _ in self.rvs]
    for index, value in enumerate(assignment):
      rv = self.rvs[index]
      indices[index] = rv.domain.index(value)
    return self.table[tuple(indices)]

  def get_by_rvs(self, rvs_to_vals):
    """Given a complete assignment of RVs to values, lookup table value.

    Args:
      rvs_to_values: A dict from RVs to values in their domains.

    Returns:
      value: The value in self.table.
    """
    assert set(rvs_to_vals.keys()) == set(self.rvs)
    indices = [None for _ in self.rvs]
    for rv, value in rvs_to_vals.items():
      index = self.rvs.index(rv)
      indices[index] = rv.domain.index(value)
    return self.table[tuple(indices)]

  def get_by_names(self, rv_name_dict):
    """Given a dict from RV names (strs) to assignments,
    return the corresponding value in the potential table.

    Args:
      rv_name_dict: A dict from str names to values.
      potential: A Potential.

    Returns:
      value: The float value from potential.table.
    """
    assert len(rv_name_dict) == len(self.rvs)
    rv_name_to_rv = {rv.name: rv for rv in self.rvs}
    rvs_to_vals = {}
    for rv_name, value in rv_name_dict.items():
      rv = rv_name_to_rv[rv_name]
      rvs_to_vals[rv] = value
    return self.get_by_rvs(rvs_to_vals)

  def __hash__(self):
    return hash(tuple(self.rvs)) ^ hash(self.table.tobytes())

  def __eq__(self, other):
    return hash(self) == hash(other)

  def __neq__(self, other):
    return not (self == other)

  def allclose(self, other, decimals=6):
    """Check whether two potentials are (nearly) equal.
    """
    if set(self.rvs) != set(other.rvs):
      raise ValueError("Can only compare potentials with the same RVs.")
    new_idxs = [other.rvs.index(rv) for rv in self.rvs]
    trans_table2 = np.transpose(other.table, new_idxs)
    assert self.table.shape == trans_table2.shape
    return np.allclose(self.table, trans_table2)


def neighbor_dict(rvs, potentials):
  """This helper function creates a mapping.
    - For each random variable rv, neighbors[rv] is a list of potentials that involve this RV.
    - For each potential pot, neighbors[pot] is a list of random variables involved in this potential.
  """
  neighbors = {v : set() for v in rvs + potentials}
  for p in potentials:
    for v in p.rvs:
      neighbors[p].add(v)
      neighbors[v].add(p)
  return neighbors


# Undirected graphical models described in 3.1 and 3.2.

def create_mrf_3_1():
  """Create the undirected graphical model (MRF) in section 3.1."""
  A, B = RV("A", [0, 1]), RV("B", [0, 1])
  rvs = [A, B]
  potentials = [
    Potential([A], np.array([1.0, 3.0])),
    Potential([B], np.array([1.0, 2.0])),
    Potential([A, B], np.array([
      [1.0, 2.0],
      [3.0, 4.0]
    ]))
  ]
  return rvs, potentials


def create_mrf_3_2():
  """Create the undirected graphical model (MRF) in section 3.2."""
  A, B, C, D = RV("A", [0, 1]), RV("B", [0, 1]), RV("C", [0, 1])
  rvs = [A, B, C, D]
  potentials = [
    Potential([A, B], np.array([
      [1.0, 2.0],
      [3.0, 4.0]
    ])),
    Potential([B, C], np.array([
      [1.0, 1.0],
      [2.0, 2.0]
    ]))
  ]
  return rvs, potentials


InferenceProblem = namedtuple("InferenceProblem",
  ["rvs", "potentials", "query", "evidence"])


# Problems for unit tests

def create_debug_2vars_problem(version):
  """A simple problem with two random variables"""
  A = RV("A", [0, 1])
  B = RV("B", [0, 1, 2])
  rvs = [A, B]
  p_a_given_b = Potential([A, B], np.array([
      [0.9, 0.15, 0.44],
      [0.1, 0.85, 0.56],
  ]))
  p_b = Potential([B], np.array([0.7, 0.2, 0.1]))
  pots = [p_a_given_b, p_b]
  if version == 1:
    query = {A : 1}
    evidence = {B : 1}
  elif version == 2:
    query = {B : 1}
    evidence = {A : 1}
  else:
    assert version == 3
    query = {A : 1, B : 1}
    evidence = {}
  return InferenceProblem(rvs, pots, query, evidence)


def create_california_problem(version):
    """Holmes, watson, earthquakes, radios, oh my...
    """
    p_b = np.array([0.99, 0.01])
    p_e = np.array([0.97, 0.03])
    p_re = np.array([
        [0.98, 0.01],
        [0.02, 0.99],
    ])
    p_aeb = np.zeros((2, 2, 2))
    p_aeb[1, 0, 0] = 0.01
    p_aeb[0, 0, 0] = 1. - 0.01
    p_aeb[1, 0, 1] = 0.2
    p_aeb[0, 0, 1] = 1. - 0.2
    p_aeb[1, 1, 0] = 0.95
    p_aeb[0, 1, 0] = 1. - 0.95
    p_aeb[1, 1, 1] = 0.96
    p_aeb[0, 1, 1] = 1. - 0.96

    A = RV("Alarm", [0, 1])
    B = RV("Burglar", [0, 1])
    E = RV("Earthquake", [0, 1])
    R = RV("Radio", [0, 1])
    rvs = [A, B, E, R]
    pots = [
        Potential([B], p_b),
        Potential([E], p_e),
        Potential([R, E], p_re),
        Potential([A, E, B], p_aeb)
    ]
    if version == "alarm":
        # P(B=1 | A=1)
        query = {B : 1}
        evidence = {A : 1}
    else:
        assert version == "alarm and earthquake"
        # P(B=1 | A=1, R=1)
        query = {B : 1}
        evidence = {A : 1, R : 1}
    return InferenceProblem(rvs, pots, query, evidence)

def create_binary_chain_problem(num_vars):
    """A simple binary chain designed to stress test inference
    """
    rvs = [RV(f"X{i}", [0, 1]) for i in range(num_vars)]
    pots = []
    for rv_t, rv_t1 in zip(rvs[:-1], rvs[1:]):
        pot = Potential([rv_t, rv_t1], np.array([
            [0.9, 0.1],
            [0.1, 0.9],
        ]))
        pots.append(pot)
    query = {rvs[0] : 0}
    evidence = {}
    return InferenceProblem(rvs, pots, query, evidence)


def iter_joint_values(rvs):
  """Iterates over joint assignments for a list of RVs.

  Returns an iterator that can be used in a for loop.

  Example usage:
    for assignment in iter_joint_values(rvs):
      print(assignment)  # a tuple
      assert assignment[0] in rvs[0].domain

  Args:
    rvs: A list of RVs.

  Yields:
    assignment: A tuple of ints representing a joint
      assignment of the random variables.
  """
  domains = [rv.domain for rv in rvs]
  return itertools.product(*domains)


def get_sub_assignment(rvs, assignment, sub_rvs):
  """Given an assignment of rvs to values, get a subassignment,
  that is, a sub-tuple of the given assignment involving only
  the given sub_rvs.

  Example usage:
    x = RV("x", [0, 1])
    y = RV("y", ["a", "b"])
    z = RV("z", [3, 5])
    rvs = (x, y, z)
    assignment = (0, "b", 3)
    sub_rvs = (z, x)
    sub_assignment = get_sub_assignment(rvs, assignment, sub_rvs)
    assert sub_assignment == (3, 0)

  Args:
    rvs: A tuple or list of RVs.
    assignment: A tuple or list of values.
    sub_rvs: A tuple or list of RVs, a subset of rvs.

  Returns:
    sub_assignment: A tuple of values.
  """
  assert set(sub_rvs).issubset(set(rvs))
  sub_assignment = []
  for rv in sub_rvs:
    idx = rvs.index(rv)
    val = assignment[idx]
    sub_assignment.append(val)
  return tuple(sub_assignment)


def run_belief_prop(problem):
  """Run inference with belief propagation.

  Calls `run_single_marginal_bp`, which is for
  you to implement.

  You will not need to use this function in your code.

  Args:
    problem: InferenceProblem
    max_iters: int
       Maximum number of iterations for each BP call.
  Returns:
    result: float
      Answer to the query in the problem
  """
  # Convert evidence to potentials
  problem = convert_evidence_to_potentials(problem)
  # Convert joint queries into a sequence of marginal queries
  problems = []
  # Set up arbitrary order for chain rule
  queries = sorted(problem.query.items(), key=lambda i: i[0].name)
  for i, (rv, val) in enumerate(queries):
    # P(rv = val | previous vals)
    new_query = {rv : val}
    new_evidence = dict(queries[:i])
    new_problem = InferenceProblem(problem.rvs,
                                   problem.potentials,
                                   new_query, new_evidence)
    new_problem = convert_evidence_to_potentials(new_problem)
    problems.append(new_problem)
  # Run individual problems
  result = 1.
  for p in problems:
    (query_rv, query_val), = p.query.items()
    p_result = run_single_marginal_bp(query_rv, query_val,
      p.rvs, p.potentials)
    result *= p_result
  return result


def convert_evidence_to_potentials(problem):
  """Create singleton potentials to account for evidence.

  Helper for run_belief_prop. You will not need to use this.
  """
  new_problem = InferenceProblem(problem.rvs,
                                 [pot for pot in problem.potentials],
                                 problem.query.copy(), {})
  for rv, val in problem.evidence.items():
    # Create singleton potential
    table = np.zeros(rv.dim)
    table[rv.domain.index(val)] = 1.
    new_potential = Potential([rv], table)
    new_problem.potentials.append(new_potential)
  return new_problem



## Problems

### Warming Up to Potentials
Write a function that returns a potential following the description in the docstring below.

For reference, our solution is **9** lines of code.

In [ ]:
def warmup():
  """Creates a potential involving two RVs.

  The RVs should be called "Rain" and "Clouds".
  Their domains should both be [0, 1].
  The potential table should have the following values:
  *   Rain=0, Clouds=0 : 0.6
  *   Rain=0, Clouds=1 : 0.15
  *   Rain=1, Clouds=0 : 0.05
  *   Rain=1, Clouds=1 : 0.2

  We are expecting your method to return a Potential. This is a class
  we have provided for you, and if you examine the colab notebook, you will
  see some documentation and some helper functions.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def warmup1_test():
  my_potential = warmup()
  assert isinstance(my_potential.rvs, (tuple, list))
  assert len(my_potential.rvs) == 2
  rain_rv, cloud_rv = None, None
  for rv in my_potential.rvs:
    assert rv.dim == 2
    if rv.name == 'Rain':
      rain_rv = rv
    elif rv.name == 'Clouds':
      cloud_rv = rv
    else:
      assert False, "Unexpected RV name"
  assert my_potential.get_by_names({'Rain': 0, 'Clouds': 0}) == 0.6
  assert my_potential.get_by_names({'Rain': 1, 'Clouds': 0}) == 0.05
  assert my_potential.get_by_names({'Rain': 0, 'Clouds': 1}) == 0.15
  assert my_potential.get_by_names({'Rain': 1, 'Clouds': 1}) == 0.2

warmup1_test()
print('Tests passed.')

### Warm Up Part 2
Write a function that queries the given potential for the specific variable values described in the docstring.

For reference, our solution is **1** lines of code.

In [ ]:
def warmup2(ab_potential):
  """Given a potential involving RVs 'A' and 'B',
  return the potential value for A = 0, B = 1.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def warmup2_test():
  A = RV("A", [0, 1])
  B = RV("B", [0, 1])
  warmup2_potential = Potential(
    rvs=[A, B],
    table=np.array([
      [0.98, 0.01],
      [0.02, 0.99],
    ])
  )
  assert warmup2(warmup2_potential) == 0.01

warmup2_test()
print('Tests passed.')

### Potential Multiplication
Write a function that multiplies a list of potentials together. (Make sure to refer to the top of the colab notebook, especially the functions `iter_joint_assignments` and `get_sub_assignment`.)

For reference, our solution is **29** lines of code.

In [ ]:
def multiply_potentials(potentials):
  """Multiply potentials together.

  Args:
    potentials: A list of Potentials.

  Returns:
    result: A new Potential.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def multiply_potentials_test1():
  A, B = RV('A', [0, 1, 2]), RV('B', [0, 1])
  # A matches B
  pot1 = Potential([A, B],
    np.array([
      [1, 0],
      [0, 1],
      [0, 0],
  ]))
  # B is 0
  pot2 = Potential([B], np.array([1, 0]))
  # A and B are both 0
  expected_result_table = np.zeros((3, 2))
  expected_result_table[0, 0] = 1
  expected_result = Potential([A, B], expected_result_table)
  result = multiply_potentials([pot1, pot2])
  assert result.allclose(expected_result)

multiply_potentials_test1()
def multiply_potentials_test2():
  A, B, C = RV('A', [0, 1]), RV('B', [0, 1]), RV('C', [0, 1])
  # A is definitely 0, B could be either
  pot1 = Potential([A, B], np.array([
    [1, 1],
    [0, 0]
  ]))
  # B is definitely 0 and C could be either
  pot2 = Potential([B, C],
    np.array([
      [1, 1],
      [0, 0]
  ]))
  # A is 0, B is 0, and C is either
  expected_result_table = np.zeros((2, 2, 2))
  expected_result_table[0, 0, :] = 1
  expected_result = Potential(
    [A, B, C],
    expected_result_table)
  result = multiply_potentials([pot1, pot2])
  assert result.allclose(expected_result)

multiply_potentials_test2()
print('Tests passed.')

### Marginalization
Write a function that marginalizes out given variables of a potential to create a new potential.

**Hint**: you may want to use the `array.sum()` function in numpy.

For reference, our solution is **11** lines of code.

In [ ]:
def marginalize(potential, rvs):
  """Create a new potential where each rv has been marginalized out.

  Args:
    potential: A Potential.
    rvs: A set of random variables in the potential.

  Returns:
    new_potential: A Potential.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def marginalize_test():
  A, B = RV('A', [0, 1]), RV('B', [0, 1, 2])
  pot = Potential([A, B],
    np.array([
      [0.54, 0.16, 0.01],
      [0.04, 0.06, 0.19],
    ]))
  expected_result = Potential([B],
    np.array([0.58, 0.22, 0.20]))
  result = marginalize(pot, {A})
  assert result.allclose(expected_result)

marginalize_test()
print('Tests passed.')

### Neighbor Relations
Given a potential and a variable attached to the potential, returns a set of the other variables attached to that potential. (Make sure your method returns a set and not a list or tuple.)

For reference, our solution is **1** lines of code.

In [ ]:
def neighbors(P, V):
  """Given a potential and a variable attached to the potential,
  returns a set of the other variables attached to that potential.

  (Make sure your method returns a set and not a list or tuple.)

  Args:
    P: A Potential.
    V: A RV.

  Returns:
    other_rvs: A set of RVs that aren't V.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def neighbors_test():
  A, B, C = RV('A', [0, 1]), RV('B', [0, 1]), RV('C', [0, 1])
  pot = Potential([A, B, C],
    np.ones((2, 2, 2)))
  assert neighbors(pot, A) == {B, C}

neighbors_test()
print('Tests passed.')

### Sum-Product 3: The Code Does Them All
Write a function that runs sum product to determine the marginal probability of a single random variable assignment.
  Feel free to use the simple graphs studied above for debugging.
  However, be sure that your code deal with potential functions that involve more than just two nodes.
  You can assume that the input undirected grahpical model is a tree.

  **Hint**: You may find the provided helper function `neighbor_dict` useful.

For reference, our solution is **18** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `marginalize`, `multiply_potentials`, `neighbors`. You may not need to use all of them.

In [ ]:
def run_single_marginal_bp(query_rv, query_val, rvs, potentials):
  """Run belief propagation on a problem with a single query
  and no evidence.

  Args:
    query_rv: A RV.
    query_val: A value in the domain of query_rv.
    rvs: All the RVs in the problem.
    potentials: All the Potentials in the problem.

  Returns:
    marginal_val: the float probability of query_rv = query_val.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def bp_test1():
  # The binary chain problem involves a single marginal
  # query with no evidence
  problem = create_binary_chain_problem(3)
  (query_rv, query_val), = problem.query.items()
  assert len(problem.evidence) == 0
  result = run_single_marginal_bp(query_rv, query_val, problem.rvs, problem.potentials)
  assert abs(result - 0.5) < 1e-5

bp_test1()
def bp_test2():
  # Run full belief propagation, which calls
  # `run_single_marginal_bp` as a subroutine
  result = run_belief_prop(create_debug_2vars_problem(1))
  assert abs(result - 0.85) < 1e-5
  result = run_belief_prop(create_debug_2vars_problem(2))
  assert abs(result - 0.574324) < 1e-5
  result = run_belief_prop(create_debug_2vars_problem(3))
  assert abs(result - 0.17) < 1e-5
  result = run_belief_prop(create_binary_chain_problem(5))
  assert abs(result - 0.5) < 1e-5
  result = run_belief_prop(create_binary_chain_problem(25))
  assert abs(result - 0.5) < 1e-5
  result = run_belief_prop(create_california_problem("alarm"))
  assert abs(result - 0.055636) < 1e-5
  result = run_belief_prop(create_california_problem("alarm and earthquake"))
  assert abs(result - 0.011386) < 1e-5

bp_test2()
print('Tests passed.')